# 1. 문서의 내용을 읽는다


In [ ]:
from docx import Document

document = Document('./tax.docx')

full_text = ''
for index, paragraph in enumerate(document.paragraphs):
    full_text += f'{paragraph.text}\n'
    

# 2. 문서를 쪼갠다

In [ ]:
%pip install tiktoken

In [ ]:
import tiktoken 

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoder.encode(full_text)
    total_token_count = len(total_encoding)
    text_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoded = encoder.decode(chunk)
        text_list.append(decoded)
    
    return text_list

In [ ]:
chunk_list = split_text(full_text, 1500)

# 3. 문서 임베딩

In [ ]:
%pip install chromadb

In [ ]:
import chromadb

chroma_client = chromadb.Client()

In [ ]:
collection_name = 'tax_collection'
tax_collection = chroma_client.create_collection(collection_name)

In [ ]:
import os

from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(api_key=openai_api_key, model_name='text-embedding-3-large')

In [ ]:
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)

In [ ]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')


In [ ]:
len(id_list)

In [ ]:
len(chunk_list)

In [ ]:
tax_collection.add(documents=chunk_list, ids=id_list)

# 4. 유사도 검색

In [ ]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_doc = tax_collection.query(query_texts=query, n_results=3)

In [ ]:
retrieved_doc['documents'][0]

# 5. LLM 질의

In [28]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요 {retrieved_doc['documents'][0]}"},
    {"role": "user", "content": query}
  ]
)

In [33]:
response.choices[0].message.content

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 먼저 소득세 과세표준과 해당 표준에 따른 세율을 알아야 합니다. 우리나라의 소득세율은 누진세 구조를 갖고 있어서, 과세표준에 따라 적용되는 세율이 다릅니다. 계산을 위한 예제는 다음과 같습니다.\n\n1. **연봉 5천만원에서 근로소득 공제를 한 후 과세표준을 계산**합니다.\n2. **연말정산 시 적용되는 세율을 적용**하여 소득세를 계산합니다.\n3. **산출된 소득세에서 근로소득세액공제를 적용**합니다.\n\n### 1. 근로소득공제 계산\n연봉 5천만원에 대한 근로소득 공제는 다음과 같이 계산됩니다.\n\n| 총급여액 (원)             | 공제액 계산 방법                    |\n|------------------------|--------------------------------|\n| 3천3백만원 초과 ~ 7천만원 이하 | 1,320만원 + (총급여액 - 3,300만원) x 25% |\n\n- **기본 공제분**: 1,320만원\n- **추가 공제분**: (5,000만원 - 3,300만원) × 25% = 1,700만원 × 25% = 425만원\n- **근로소득공제 합계**: 1,320만원 + 425만원 = 1,745만원\n\n### 2. 과세표준 계산\n과세표준은 총급여액에서 근로소득공제액을 뺀 금액입니다.\n\n과세표준 = 연봉 5천만원 - 1,745만원 = 3,255만원\n\n### 3. 기본 세율 적용에 의한 소득세 계산\n과세표준 3,255만원에 대응하는 세율을 적용합니다.\n\n| 과세표준 (원)              | 세율  |\n|-----------------------|------|\n| 1,200만원 초과 ~ 4,600만원 이하 | 15%  |\n\n과세표준이 3,255만원이므로 해당 금액이 위치하는 세율은 15%입니다.\n\n- **기본 공제분**: 1,200만원 x 6% = 72만원\n- **추가 공제분**: (3,255만원 - 1,200만원)